In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import time

In [2]:
"""PyTorch Hub models
Wanted Usage:
    import torch
    model = torch.hub.load('repo', 'model')
"""

from pathlib import Path

import torch

from models.yolo import Model
from utils.general import check_requirements, set_logging
from utils.google_utils import attempt_download
from utils.torch_utils import select_device

dependencies = ['torch', 'yaml']
set_logging()


def create(name, pretrained, channels, classes, autoshape):
    """Creates a specified model
    Arguments:
        name (str): name of model, i.e. 'yolov7'
        pretrained (bool): load pretrained weights into the model
        channels (int): number of input channels
        classes (int): number of model classes
    Returns:
        pytorch model
    """
    try:
        cfg = list((Path(__file__).parent / 'cfg').rglob(f'{name}.yaml'))[0]  # model.yaml path
        model = Model(cfg, channels, classes)
        if pretrained:
            fname = f'{name}.pt'
            attempt_download(fname)
            ckpt = torch.load(fname, map_location=torch.device('cpu'))
            msd = model.state_dict()  # model state_dict
            csd = ckpt['model'].float().state_dict()
            csd = {k: v for k, v in csd.items() if msd[k].shape == v.shape}
            model.load_state_dict(csd, strict=False)
            if len(ckpt['model'].names) == classes:
                model.names = ckpt['model'].names
            if autoshape:
                model = model.autoshape()
        device = select_device('0' if torch.cuda.is_available() else 'cpu')  # default to GPU
        return model.to(device)

    except Exception as e:
        s = 'Cache maybe be out of date, try force_reload=True.'
        raise Exception(s) from e


def custom(path_or_model='path/to/model.pt', autoshape=True):
    """custom mode
    Arguments (3 options):
        path_or_model (str): 'path/to/model.pt'
        path_or_model (dict): torch.load('path/to/model.pt')
        path_or_model (nn.Module): torch.load('path/to/model.pt')['model']
    Returns:
        pytorch model
    """
    model = torch.load(path_or_model, map_location=torch.device('cpu')) if isinstance(path_or_model, str) else path_or_model  # load checkpoint
    if isinstance(model, dict):
        model = model['ema' if model.get('ema') else 'model']

    hub_model = Model(model.yaml).to(next(model.parameters()).device)
    hub_model.load_state_dict(model.float().state_dict())
    hub_model.names = model.names
    if autoshape:
        hub_model = hub_model.autoshape()
    device = select_device('0' if torch.cuda.is_available() else 'cpu')  # default to GPU if available
    return hub_model.to(device)


def yolov7(pretrained=True, channels=3, classes=80, autoshape=True):
    return create('yolov7', pretrained, channels, classes, autoshape)


if __name__ == '__main__':
    model = custom(path_or_model='yolov7_custom.pt')  # custom example
    # model = create(name='yolov7', pretrained=True, channels=3, classes=80, autoshape=True)  # pretrained example

    # Verify inference
    import numpy as np
    from PIL import Image

    imgs = [np.zeros((640, 480, 3))]
    img = os.path.join('data', 'train', 'images', 'b1WIN_20221125_15_27_37_Pro.jpg')

    results = model(img)  # batched inference
    #results.print()
    #results.save()
    print(results.pandas().xyxy[0].name.to_json())


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

 76                -1  1         0  models.common.MP                        []                            
 77                -1  1     16640  models.common.Conv                      [128, 128, 1, 1]              
 78                -3  1     16640  models.common.Conv                      [128, 128, 1, 1]              
 79                -1  1    147712  models.common.Conv                      [128, 128, 3, 2]              
 80      [-1, -3, 63]  1         0  models.common.Concat                    [1]                           
 81                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]              
 82                -2  1    131584  models.common.Conv                      [512, 256, 1, 1]              
 83                -1  1    295168  models.common.Conv                      [256, 128, 3, 1]              
 84                -1  1    147712  models.common.Conv                      [128, 128, 3, 1]              
 85                -1  1    147712  m

Adding autoShape... 
{"0":"center","1":"center"}


In [ ]:
import serial
ser = serial.Serial("COM5", 9600)
print("ser worked")
cap = cv2.VideoCapture(0) #"gozlu.mp4"
print("cam workde")
while cap.isOpened():
    ret, frame = cap.read()
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    #x = results.pandas().xyxy[0].name.to_json()
    max = 0
    max_index = 0
    i=0
    for x in results.pandas().xyxy[0].confidence:
        if(x > max):
            max = x
            max_index = i
        i += 1
        
    
    if(i==0 or max <= 0.74):
        ser.write(b'7') #unknown
    else:
        if(str(results.pandas().xyxy[0].name[max_index]) == "center"):
            ser.write(b'9')
            time.sleep(0.2)
        if(str(results.pandas().xyxy[0].name[max_index]) == "left"):
            ser.write(b'3')
            time.sleep(0.7)
        if(str(results.pandas().xyxy[0].name[max_index]) == "right"):
            ser.write(b'4')
            time.sleep(0.7)
        if(str(results.pandas().xyxy[0].name[max_index]) == "up"):
            ser.write(b'1')
            time.sleep(0.5)
        elif(str(results.pandas().xyxy[0].name[max_index]) == "down"):
            ser.write(b'2')
            time.sleep(0.5)
        elif(str(results.pandas().xyxy[0].name[max_index]) == "closed"):
            ser.write(b'5')
            time.sleep(1)
            
            

    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
ser.close()

In [ ]:
ser.close()